# Sumarização com similaridade do cosseno

## Preparação do texto de exemplo

In [37]:
import re
import nltk
import spacy
import heapq
import string

import numpy    as np
import networkx as nx

from goose3      import Goose
from collections import Counter

from nltk.cluster.util    import cosine_distance
from IPython.core.display import HTML

In [2]:
nlp = spacy.load('pt_core_news_sm')

In [3]:
doc = nlp("A inteligência artificial é a inteligência similar à humana máquinas.")
doc[-1].is_punct

True

In [4]:
def preprocess_sentence(text):
    
    doc = nlp(text.lower())
    
    tokens = [token.lemma_ for token in doc if token.is_stop == False and token.is_punct == False]
    
    return ' '.join(tokens)

In [5]:
preprocess_sentence("A inteligência artificial é a inteligência similar à humana máquinas.")

'inteligência artificial inteligência similar humano máquina'

In [6]:
original_text = """A inteligência artificial é a inteligência similar à humana máquinas.
                    Definem como o estudo de agente artificial com inteligência.
                    Ciência e engenharia de produzir máquinas com inteligência.
                    Resolver problemas e possuir inteligência.
                    Relacionada ao comportamento inteligente.
                    Construção de máquinas para raciocinar.
                    Aprender com os erros e acertos.
                    Inteligência artificial é raciocinar nas situações do cotidiano."""

original_text = re.sub(r'\s+', ' ', original_text)
original_text = re.split(r'\.', original_text, re.DOTALL)
original_text = [re.sub(r'^\s', '', s, re.MULTILINE) for s in original_text[:-1]]

original_text

['A inteligência artificial é a inteligência similar à humana máquinas',
 'Definem como o estudo de agente artificial com inteligência',
 'Ciência e engenharia de produzir máquinas com inteligência',
 'Resolver problemas e possuir inteligência',
 'Relacionada ao comportamento inteligente',
 'Construção de máquinas para raciocinar',
 'Aprender com os erros e acertos',
 'Inteligência artificial é raciocinar nas situações do cotidiano']

## Função para calcular a similaridade entre sentenças

- Link: https://en.wikipedia.org/wiki/Cosine_similarity
- Cálculos passo a passo: https://janav.wordpress.com/2013/10/27/tf-idf-and-cosine-similarity/

In [7]:
preprocessed_sentences = [preprocess_sentence(s) for s in original_text]
preprocessed_sentences

['inteligência artificial inteligência similar humano máquina',
 'definir estudo agente artificial inteligência',
 'ciência engenharia produzir máquina inteligência',
 'resolver problema possuir inteligência',
 'relacionar comportamento inteligente',
 'construção máquina raciocinar',
 'aprender erro acerto',
 'inteligência artificial raciocinar situação cotidiano']

In [11]:
def calc_sentence_similarity(sentence_one, sentence_two):
    
    list_words_one = nltk.word_tokenize(sentence_one)
    list_words_two = nltk.word_tokenize(sentence_two)
    
    list_all_words = {word: i for i, word in enumerate(set(list_words_one + list_words_two))}
    
    counter_one = Counter(list_words_one)
    counter_two = Counter(list_words_two)
    
    array_one = np.array([counter_one.get(word, 0) for word in list_all_words], dtype=int)
    array_two = np.array([counter_two.get(word, 0) for word in list_all_words], dtype=int)
    
    return 1 - cosine_distance(array_one, array_two)

In [12]:
nltk.word_tokenize(preprocessed_sentences[0])

['inteligência', 'artificial', 'inteligência', 'similar', 'humano', 'máquina']

In [15]:
calc_similarity(preprocessed_sentences[0], preprocessed_sentences[0])

np.float64(0.9999999999999998)

## Função para gerar a matriz de similaridade

In [21]:
def calc_similarity_matrix(sentences):
    
    num_sentences = len(sentences)
    
    similarity_matrix = np.zeros((num_sentences, num_sentences))

    for i, sentence_one in enumerate(sentences):
        
        for j in range(i + 1, num_sentences):
            
            similarity = calc_similarity(sentence1, sentences[j])
            
            similarity_matrix[i, j] = similarity
            similarity_matrix[j, i] = similarity

    return similarity_matrix

In [20]:
calcula_matriz_similaridade(preprocessed_sentences)

array([[0.        , 0.47434165, 0.47434165, 0.35355339, 0.        ,
        0.20412415, 0.        , 0.47434165],
       [0.47434165, 0.        , 0.2       , 0.2236068 , 0.        ,
        0.        , 0.        , 0.4       ],
       [0.47434165, 0.2       , 0.        , 0.2236068 , 0.        ,
        0.25819889, 0.        , 0.2       ],
       [0.35355339, 0.2236068 , 0.2236068 , 0.        , 0.        ,
        0.        , 0.        , 0.2236068 ],
       [0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        ],
       [0.20412415, 0.        , 0.25819889, 0.        , 0.        ,
        0.        , 0.        , 0.25819889],
       [0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        ],
       [0.47434165, 0.4       , 0.2       , 0.2236068 , 0.        ,
        0.25819889, 0.        , 0.        ]])

## Função para sumarizar

- Algoritmo Page Rank: https://www.youtube.com/watch?v=YfDNI1jp5sM e https://www.youtube.com/watch?v=YplmCue8XJU

In [49]:
nltk.sent_tokenize("""A inteligência artificial é a inteligência similar à humana máquinas.
                    Definem como o estudo de agente artificial com inteligência.
                    Ciência e engenharia de produzir máquinas com inteligência.
                    Resolver problemas e possuir inteligência.
                    Relacionada ao comportamento inteligente.
                    Construção de máquinas para raciocinar.
                    Aprender com os erros e acertos.
                    Inteligência artificial é raciocinar nas situações do cotidiano.""")

['A inteligência artificial é a inteligência similar à humana máquinas.',
 'Definem como o estudo de agente artificial com inteligência.',
 'Ciência e engenharia de produzir máquinas com inteligência.',
 'Resolver problemas e possuir inteligência.',
 'Relacionada ao comportamento inteligente.',
 'Construção de máquinas para raciocinar.',
 'Aprender com os erros e acertos.',
 'Inteligência artificial é raciocinar nas situações do cotidiano.']

In [15]:
def sumarize(text, n_sentences):

    sentences = [sentence for sentence in nltk.sent_tokenize(text)]

    preprocessed_sentences = [preprocess_sentence(s) for s in sentences]

    matrix_similarity = calc_similarity_matrix(preprocessed_sentences)

    similarity_graph = nx.from_numpy_array(matrix_similarity)

    scores = nx.pagerank(similarity_graph)

    sorted_scores = sorted(((scores[i], score) for i, score in enumerate(sentences)), reverse=True)

    list_best_sentences = [sorted_scores[i][1] for i in range(n_sentences)]

    return sentences, list_best_sentences, sorted_scores

In [16]:
sumarize("""A inteligência artificial é a inteligência similar à humana máquinas.
                    Definem como o estudo de agente artificial com inteligência.
                    Ciência e engenharia de produzir máquinas com inteligência.
                    Resolver problemas e possuir inteligência.
                    Relacionada ao comportamento inteligente.
                    Construção de máquinas para raciocinar.
                    Aprender com os erros e acertos.
                    Inteligência artificial é raciocinar nas situações do cotidiano.""", 3)

(['A inteligência artificial é a inteligência similar à humana máquinas.',
  'Definem como o estudo de agente artificial com inteligência.',
  'Ciência e engenharia de produzir máquinas com inteligência.',
  'Resolver problemas e possuir inteligência.',
  'Relacionada ao comportamento inteligente.',
  'Construção de máquinas para raciocinar.',
  'Aprender com os erros e acertos.',
  'Inteligência artificial é raciocinar nas situações do cotidiano.'],
 ['A inteligência artificial é a inteligência similar à humana máquinas.',
  'Inteligência artificial é raciocinar nas situações do cotidiano.',
  'Ciência e engenharia de produzir máquinas com inteligência.'],
 [(0.22820924040178003,
   'A inteligência artificial é a inteligência similar à humana máquinas.'),
  (0.1839190802151221,
   'Inteligência artificial é raciocinar nas situações do cotidiano.'),
  (0.1633191450783496,
   'Ciência e engenharia de produzir máquinas com inteligência.'),
  (0.1543717033327776,
   'Definem como o estudo

## Criação da Classe

In [59]:
class CosineDistanceSummarizer():

    def __init__(self, text, n_sentences, spacy_model="pt_core_news_sm"):

        self.text = text
        
        self.n_sentences = n_sentences

        self.spacy_model = spacy_model

    def _tokenize(self):

        return nltk.sent_tokenize(self.text)

    def _preprocess_sentences(self, sentences):

        nlp = spacy.load(self.spacy_model)
        
        return [
            " ".join(
                token.lemma_ 
                for token in nlp(sentence.lower())
                if not token.is_stop and not token.is_punct
            )
            for sentence in sentences
        ]

    def _calc_similarity_matrix(self, sentences):
    
        num_sentences = len(sentences)
    
        similarity_matrix = np.zeros((num_sentences, num_sentences))

        for i, sentence_one in enumerate(sentences):
        
            for j in range(i + 1, num_sentences):
            
                similarity = calc_similarity(sentence_one, sentences[j])
            
                similarity_matrix[i, j] = similarity
                similarity_matrix[j, i] = similarity

        return similarity_matrix

    def _calc_pagerank(self, similarity_matrix):

        similarity_graph = nx.from_numpy_array(similarity_matrix)
        
        scores = nx.pagerank(similarity_graph)

        return scores

    def summarize(self):

        self.list_sentences = self._tokenize()

        list_preprocessed_sentences = self._preprocess_sentences(self.list_sentences)

        similarity_matrix = self._calc_similarity_matrix(list_preprocessed_sentences)

        self.scores = self._calc_pagerank(similarity_matrix)
        
        self.sorted_scores = [(self.scores[i], sentence) for i, sentence in enumerate(self.list_sentences)]
    
        list_best_sentences = [sentence for _, sentence in heapq.nlargest(self.n_sentences, self.sorted_scores)]
        
        return list_best_sentences

In [64]:
s = CosineDistanceSummarizer("""A inteligência artificial é a inteligência similar à humana máquinas.
                    Definem como o estudo de agente artificial com inteligência.
                    Ciência e engenharia de produzir máquinas com inteligência.
                    Resolver problemas e possuir inteligência.
                    Relacionada ao comportamento inteligente.
                    Construção de máquinas para raciocinar.
                    Aprender com os erros e acertos.
                    Inteligência artificial é raciocinar nas situações do cotidiano.""", 3)

In [65]:
s.summarize()

['A inteligência artificial é a inteligência similar à humana máquinas.',
 'Inteligência artificial é raciocinar nas situações do cotidiano.',
 'Ciência e engenharia de produzir máquinas com inteligência.']

In [66]:
s.list_sentences

['A inteligência artificial é a inteligência similar à humana máquinas.',
 'Definem como o estudo de agente artificial com inteligência.',
 'Ciência e engenharia de produzir máquinas com inteligência.',
 'Resolver problemas e possuir inteligência.',
 'Relacionada ao comportamento inteligente.',
 'Construção de máquinas para raciocinar.',
 'Aprender com os erros e acertos.',
 'Inteligência artificial é raciocinar nas situações do cotidiano.']

In [67]:
s.scores

{0: 0.22820924040178003,
 1: 0.1543717033327776,
 2: 0.1633191450783496,
 3: 0.12639273963873288,
 4: 0.023809527846794958,
 5: 0.09616903563964783,
 6: 0.023809527846794958,
 7: 0.1839190802151221}

In [68]:
s.sorted_scores

[(0.22820924040178003,
  'A inteligência artificial é a inteligência similar à humana máquinas.'),
 (0.1543717033327776,
  'Definem como o estudo de agente artificial com inteligência.'),
 (0.1633191450783496,
  'Ciência e engenharia de produzir máquinas com inteligência.'),
 (0.12639273963873288, 'Resolver problemas e possuir inteligência.'),
 (0.023809527846794958, 'Relacionada ao comportamento inteligente.'),
 (0.09616903563964783, 'Construção de máquinas para raciocinar.'),
 (0.023809527846794958, 'Aprender com os erros e acertos.'),
 (0.1839190802151221,
  'Inteligência artificial é raciocinar nas situações do cotidiano.')]

## Visualização do resumo

In [69]:
def visualiza_resumo(titulo, lista_sentencas, melhores_sentencas):
  
    texto = ''

    display(HTML(f'<h1>Resumo do texto - {titulo}</h1>'))
  
    for i in lista_sentencas:
        
        if i in melhores_sentencas:
            texto += str(i).replace(i, f"<mark>{i}</mark>")
    
        else:
            texto += i
    
    display(HTML(f""" {texto} """))

In [70]:
texto_original = """A inteligência artificial é a inteligência similar à humana máquinas.
                    Definem como o estudo de agente artificial com inteligência.
                    Ciência e engenharia de produzir máquinas com inteligência.
                    Resolver problemas e possuir inteligência.
                    Relacionada ao comportamento inteligente.
                    Construção de máquinas para raciocinar.
                    Aprender com os erros e acertos.
                    Inteligência artificial é raciocinar nas situações do cotidiano."""

resumo_texto = CosineDistanceSummarizer(texto_original, 3).summarize()

In [71]:
visualiza_resumo('Teste', texto_original, resumo_texto)

## Extração de texto da internet

In [72]:
g = Goose()
url = 'https://iaexpert.academy/2020/11/09/ia-preve-resultado-das-eleicoes-americanas/'
artigo = g.extract(url)

In [73]:
artigo.cleaned_text

'Nas eleições presidenciais americanas de 2016, a maioria das predições apontavam para a vitória de Hillary Clinton. Entretanto, a história nos mostrou o resultado oposto, e Donald Trump foi o presidente nos últimos 4 anos. Desta vez, os estatísticos reexaminaram seus modelos, para aumentar o grau de confiabilidade nos seus resultados. Nesta tentativa de otimização das predições, a inteligência artificial certamente não ficou de fora.\n\nO modelo desenvolvido pelo Dr. Hernan Makse, físico estatístico da Universidade da Cidade de Nova York, baseou suas predições em uma rede neural treinada para processar os sentimentos expressos nas redes sociais. O algoritmo fez a análise de cerca de 1 bilhão de tweets para chegar a uma estimativa dos resultados do pleito. No dia da eleição, 3 de novembro, o modelo estava indicando a vitória de Joe Biden.\n\nO Dr. Makse disse que seu trabalho começou já na eleição de 2016, e foi testado novamente nas eleições na Argentina ano passado. Desta vez, o mode

In [74]:
melhores_sentencas = CosineDistanceSummarizer(artigo.cleaned_text, 5).summarize()

In [75]:
visualiza_resumo('Teste', artigo.cleaned_text, melhores_sentencas)